# CST3133 - Traditional Machine Learning Workflow and Ethics
## CSGO Match Prediction Dataset

This notebook implements the complete machine learning workflow for predicting CSGO match outcomes using structured data analysis.

## 1.1.1 Dataset Selection and Problem Definition (5%)

### Dataset Overview
We are using a **CSGO (Counter-Strike: Global Offensive) match dataset** that contains historical match data with team statistics and player performance metrics.

**Dataset Source:** Publicly available CSGO match dataset
**Dataset Link:** [To be provided in report]

### Problem Definition
**Problem Type:** **Classification Problem**

We aim to predict the **match winner** (Team 1 or Team 2) based on:
- Team rankings and historical performance
- Individual player statistics (ratings, KDR, impact, etc.)
- Head-to-head win percentages
- Match points

**Target Variable:** `winner` (categorical: 't1' or 't2')

**Features:** 
- Team-level features: rankings, points, head-to-head statistics
- Player-level features: ratings, KDR (Kill-Death Ratio), impact, damage, assists, etc. for all 10 players (5 per team)

### Environment Setup

**For Google Colab (Recommended):**
- No virtual environment needed! Google Colab comes with all required packages pre-installed.
- Simply upload this notebook and the `csgo_games.csv` file to Google Colab.
- If any package is missing, uncomment and run the cell below.

**For Local Development (Optional):**
- If running locally, you can create a virtual environment:
  ```bash
  python -m venv venv
  source venv/bin/activate  # On Windows: venv\Scripts\activate
  pip install pandas numpy matplotlib seaborn scikit-learn
  ```

In [ ]:
# Install packages if needed (uncomment if running in Google Colab and packages are missing)
# !pip install pandas numpy matplotlib seaborn scikit-learn
# !pip install xgboost  # Optional: For advanced boosting models (will be checked automatically)
# !pip install lightgbm  # Optional: Alternative gradient boosting (will be checked automatically)

# Note: This cell should be run AFTER the imports cell below
# Verification will happen after imports

In [9]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Verify installations
import sys
print(f"Python version: {sys.version.split()[0]}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
try:
    import sklearn
    print(f"Scikit-learn version: {sklearn.__version__}")
except:
    print("Scikit-learn: Installed")
print("\n✓ All required packages are available!")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Load the dataset
df = pd.read_csv('csgo_games.csv')

# Display basic information
print("Dataset Shape:", df.shape)
print("\nColumn Names:")
print(df.columns.tolist())
print("\nFirst few rows:")
df.head()

In [ ]:
# Dataset overview
print("Dataset Info:")
df.info()
print("\n" + "="*50)
print("Target Variable Distribution:")
print(df['winner'].value_counts())
print("\n" + "="*50)
print("Basic Statistics:")
df.describe()

## 1.1.2 Data Preprocessing (10%)

### Step 1: Handle Missing Values
We will identify and handle missing values appropriately.

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing Count': missing_values.values,
    'Percentage': missing_percentage.values
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

print("Missing Values Summary:")
print(missing_df)

# Visualize missing values
if len(missing_df) > 0:
    plt.figure(figsize=(12, 6))
    plt.barh(missing_df['Column'], missing_df['Percentage'])
    plt.xlabel('Missing Percentage (%)')
    plt.title('Missing Values by Column')
    plt.tight_layout()
    plt.show()

In [ ]:
# Introduce noise to the dataset (as per requirements)
# We'll introduce random NaN values to some columns to demonstrate noise handling
np.random.seed(42)
df_noisy = df.copy()

# Select some numerical columns to introduce noise
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# Exclude target variable and key identifiers
cols_to_noise = [col for col in numerical_cols if col not in ['t1_points', 't2_points']]

# Introduce 5% random missing values to selected columns
for col in cols_to_noise[:10]:  # Add noise to first 10 numerical columns
    n_missing = int(len(df_noisy) * 0.05)
    missing_indices = np.random.choice(df_noisy.index, n_missing, replace=False)
    df_noisy.loc[missing_indices, col] = np.nan

print("Noise introduced. New missing values:")
print(df_noisy.isnull().sum().sum(), "total missing values")
print("\nColumns with introduced noise:")
noisy_cols = df_noisy.isnull().sum()[df_noisy.isnull().sum() > 0].index.tolist()
print(noisy_cols)

In [ ]:
# Handle missing values
# For numerical columns: use median imputation
# For categorical columns: use mode imputation

df_processed = df_noisy.copy()

# Separate numerical and categorical columns
numerical_cols = df_processed.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df_processed.select_dtypes(include=['object', 'bool']).columns.tolist()

# Impute numerical columns with median
numerical_imputer = SimpleImputer(strategy='median')
df_processed[numerical_cols] = numerical_imputer.fit_transform(df_processed[numerical_cols])

# Impute categorical columns with mode
for col in categorical_cols:
    if df_processed[col].isnull().sum() > 0:
        mode_value = df_processed[col].mode()[0] if len(df_processed[col].mode()) > 0 else 'Unknown'
        df_processed[col].fillna(mode_value, inplace=True)

print("Missing values after imputation:", df_processed.isnull().sum().sum())
print("\nImputation Strategy:")
print("- Numerical columns: Median imputation")
print("- Categorical columns: Mode imputation")

### Step 2: Feature Engineering and Encoding
We will create logical features and encode categorical variables.

In [ ]:
# Feature Engineering: Create aggregated team statistics
df_features = df_processed.copy()

# Calculate average team ratings
t1_rating_cols = [col for col in df_features.columns if 't1_player' in col and 'rating' in col]
t2_rating_cols = [col for col in df_features.columns if 't2_player' in col and 'rating' in col]

df_features['t1_avg_rating'] = df_features[t1_rating_cols].mean(axis=1)
df_features['t2_avg_rating'] = df_features[t2_rating_cols].mean(axis=1)

# Calculate average team impact
t1_impact_cols = [col for col in df_features.columns if 't1_player' in col and 'impact' in col]
t2_impact_cols = [col for col in df_features.columns if 't2_player' in col and 'impact' in col]

df_features['t1_avg_impact'] = df_features[t1_impact_cols].mean(axis=1)
df_features['t2_avg_impact'] = df_features[t2_impact_cols].mean(axis=1)

# Calculate average team KDR
t1_kdr_cols = [col for col in df_features.columns if 't1_player' in col and 'kdr' in col]
t2_kdr_cols = [col for col in df_features.columns if 't2_player' in col and 'kdr' in col]

df_features['t1_avg_kdr'] = df_features[t1_kdr_cols].mean(axis=1)
df_features['t2_avg_kdr'] = df_features[t2_kdr_cols].mean(axis=1)

# Rank difference (lower rank number = better team)
df_features['rank_diff'] = df_features['t1_world_rank'] - df_features['t2_world_rank']

# Points difference - REMOVED (data leakage: points are known after match)
# df_features['points_diff'] = df_features['t1_points'] - df_features['t2_points']

# H2H advantage
df_features['h2h_advantage'] = df_features['t1_h2h_win_perc'] - df_features['t2_h2h_win_perc']

print("New engineered features created:")
new_features = ['t1_avg_rating', 't2_avg_rating', 't1_avg_impact', 't2_avg_impact', 
                't1_avg_kdr', 't2_avg_kdr', 'rank_diff', 'h2h_advantage']
print(new_features)
print("\nSample of engineered features:")
df_features[new_features].head()

In [ ]:
# Prepare features for modeling
# First, check and filter the winner column to ensure binary classification
print("Original winner column unique values:", df_features['winner'].unique())
print("Original winner column value counts:")
print(df_features['winner'].value_counts())

# Filter to only keep 't1' and 't2' if there are other values
df_features = df_features[df_features['winner'].isin(['t1', 't2'])].copy()
print(f"\nAfter filtering: {len(df_features)} rows remaining")

# Select relevant features (exclude identifiers and target)
exclude_cols = ['match_date', 'team_1', 'team_2', 'winner']
feature_cols = [col for col in df_features.columns if col not in exclude_cols]

# Separate features and target
X = df_features[feature_cols]
y = df_features['winner']

# Encode categorical variables
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Handle boolean columns (convert to int)
bool_cols = X.select_dtypes(include=['bool']).columns
X[bool_cols] = X[bool_cols].astype(int)

# Ensure all features are numerical
X = X.select_dtypes(include=[np.number])

# Remove data leakage features (these contain information only known after the match)
leakage_features = ['t1_points', 't2_points', 'points_diff']
X = X.drop(columns=[col for col in leakage_features if col in X.columns], errors='ignore')
print(f"Removed data leakage features. Remaining features: {X.shape[1]}")
print(f"Removed features: {[col for col in leakage_features if col in df_features.columns]}")






print(f"Feature matrix shape: {X.shape}")
print(f"Target variable shape: {y_encoded.shape}")
print(f"\nTarget encoding: {dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))}")

In [ ]:
# Scale/Normalize numerical features
scaler = StandardScaler()
X_scaled = pd.DataFrame(
    scaler.fit_transform(X),
    columns=X.columns,
    index=X.index
)

print("Features scaled using StandardScaler")
print(f"Scaled features shape: {X_scaled.shape}")
print("\nSample of scaled features:")
X_scaled.head()

## 1.1.3 Exploratory Data Analysis (10%)

### Visualizations and Pattern Analysis

In [ ]:
# Target variable distribution
plt.figure(figsize=(10, 6))
y.value_counts().plot(kind='bar', color=['skyblue', 'lightcoral'])
plt.title('Distribution of Match Winners')
plt.xlabel('Winner')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("Winner Distribution:")
print(y.value_counts())
print(f"\nClass Balance: {y.value_counts(normalize=True)}")

In [ ]:
# Histograms of key features
key_features = ['t1_world_rank', 't2_world_rank', 't1_avg_rating', 't2_avg_rating', 
                't1_avg_kdr', 't2_avg_kdr', 'rank_diff', 'h2h_advantage']

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.ravel()

for i, feature in enumerate(key_features):
    if feature in df_features.columns:
        axes[i].hist(df_features[feature], bins=30, edgecolor='black', alpha=0.7)
        axes[i].set_title(f'Distribution of {feature}')
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel('Frequency')
        axes[i].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
# Select a subset of features for correlation analysis
# Note: Removed t1_points, t2_points, points_diff (data leakage)
correlation_features = ['t1_world_rank', 't2_world_rank',
                        't1_h2h_win_perc', 't2_h2h_win_perc', 't1_avg_rating', 't2_avg_rating',
                        't1_avg_kdr', 't2_avg_kdr', 'rank_diff', 'h2h_advantage']

correlation_features = [f for f in correlation_features if f in df_features.columns]

plt.figure(figsize=(12, 10))
correlation_matrix = df_features[correlation_features].corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Heatmap of Key Features')
plt.tight_layout()
plt.show()

print("\nKey Correlations:")
print("Strong positive correlations indicate features that move together.")
print("Strong negative correlations indicate inverse relationships.")

In [ ]:
# Analyze relationship between rank difference and winner
if 'rank_diff' in df_features.columns:
    plt.figure(figsize=(12, 6))
    
    # Box plot
    plt.subplot(1, 2, 1)
    df_features.boxplot(column='rank_diff', by='winner', ax=plt.gca())
    plt.title('Rank Difference by Winner')
    plt.suptitle('')  # Remove default title
    plt.xlabel('Winner')
    plt.ylabel('Rank Difference (t1_rank - t2_rank)')
    
    # Violin plot
    plt.subplot(1, 2, 2)
    sns.violinplot(data=df_features, x='winner', y='rank_diff')
    plt.title('Rank Difference Distribution by Winner')
    plt.xlabel('Winner')
    plt.ylabel('Rank Difference')
    
    plt.tight_layout()
    plt.show()
    
    print("\nInsight: Negative rank_diff means Team 1 has better rank (lower number).")
    print("Positive rank_diff means Team 2 has better rank.")

In [ ]:
# Analyze average ratings by winner
if 't1_avg_rating' in df_features.columns and 't2_avg_rating' in df_features.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Team 1 average rating by winner
    axes[0].scatter(df_features['t1_avg_rating'], 
                   [1 if w == 't1' else 0 for w in df_features['winner']], 
                   alpha=0.5, label='Team 1 Wins')
    axes[0].scatter(df_features['t1_avg_rating'], 
                   [1 if w == 't2' else 0 for w in df_features['winner']], 
                   alpha=0.5, label='Team 2 Wins')
    axes[0].set_xlabel('Team 1 Average Rating')
    axes[0].set_ylabel('Winner (1=t1, 0=t2)')
    axes[0].set_title('Team 1 Rating vs Winner')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Team 2 average rating by winner
    axes[1].scatter(df_features['t2_avg_rating'], 
                   [1 if w == 't2' else 0 for w in df_features['winner']], 
                   alpha=0.5, label='Team 2 Wins')
    axes[1].scatter(df_features['t2_avg_rating'], 
                   [1 if w == 't1' else 0 for w in df_features['winner']], 
                   alpha=0.5, label='Team 1 Wins')
    axes[1].set_xlabel('Team 2 Average Rating')
    axes[1].set_ylabel('Winner (1=t2, 0=t1)')
    axes[1].set_title('Team 2 Rating vs Winner')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

### Key Trends and Patterns

1. **Class Distribution**: The dataset shows [describe balance/imbalance]
2. **Rank Impact**: Teams with better rankings (lower numbers) tend to win more often
3. **Rating Correlation**: Higher average player ratings correlate with match wins
4. **Feature Relationships**: [Describe key correlations found]

### Potential Biases

1. **Temporal Bias**: Matches from different time periods may have different meta-games
2. **Team Representation**: Some teams may be over/under-represented
3. **Rank Bias**: Rankings may not reflect current team strength accurately

## 1.1.4 Model Development and Evaluation (15%)

### Train-Test Split

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"\nTraining set class distribution:")
print(pd.Series(y_train).value_counts())
print(f"\nTest set class distribution:")
print(pd.Series(y_test).value_counts())

### Model 1: Logistic Regression

In [ ]:
# Train Logistic Regression
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train, y_train)

# Predictions
y_pred_lr = lr_model.predict(X_test)
y_pred_proba_lr = lr_model.predict_proba(X_test)[:, 1]

# Check unique values to ensure binary classification
print(f"Unique values in y_test: {np.unique(y_test)}")
print(f"Unique values in y_pred_lr: {np.unique(y_pred_lr)}")
print(f"Number of classes: {len(np.unique(y_test))}")

# Evaluation - use appropriate average based on number of classes
if len(np.unique(y_test)) > 2:
    # Multiclass: use 'macro' average
    lr_accuracy = accuracy_score(y_test, y_pred_lr)
    lr_precision = precision_score(y_test, y_pred_lr, average='macro', zero_division=0)
    lr_recall = recall_score(y_test, y_pred_lr, average='macro', zero_division=0)
    lr_f1 = f1_score(y_test, y_pred_lr, average='macro', zero_division=0)
    print("\n⚠️  Note: Using multiclass metrics (macro average) because target has more than 2 classes")
else:
    # Binary classification
    lr_accuracy = accuracy_score(y_test, y_pred_lr)
    lr_precision = precision_score(y_test, y_pred_lr, average='binary', zero_division=0)
    lr_recall = recall_score(y_test, y_pred_lr, average='binary', zero_division=0)
    lr_f1 = f1_score(y_test, y_pred_lr, average='binary', zero_division=0)

print("\nLogistic Regression Results:")
print(f"Accuracy: {lr_accuracy:.4f}")
print(f"Precision: {lr_precision:.4f}")
print(f"Recall: {lr_recall:.4f}")
print(f"F1-Score: {lr_f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr, target_names=['Team 2', 'Team 1']))

In [ ]:
# Confusion Matrix for Logistic Regression
cm_lr = confusion_matrix(y_test, y_pred_lr)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Team 2', 'Team 1'], 
            yticklabels=['Team 2', 'Team 1'])
plt.title('Confusion Matrix - Logistic Regression')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

### Model 2: Decision Tree

In [ ]:
# Train Decision Tree
dt_model = DecisionTreeClassifier(random_state=42, max_depth=10, min_samples_split=20)
dt_model.fit(X_train, y_train)

# Predictions
y_pred_dt = dt_model.predict(X_test)

# Check unique values
print(f"Unique values in y_test: {np.unique(y_test)}")
print(f"Unique values in y_pred_dt: {np.unique(y_pred_dt)}")
print(f"Number of classes: {len(np.unique(y_test))}")

# Evaluation - use appropriate average based on number of classes
if len(np.unique(y_test)) > 2:
    # Multiclass: use 'macro' average
    dt_accuracy = accuracy_score(y_test, y_pred_dt)
    dt_precision = precision_score(y_test, y_pred_dt, average='macro', zero_division=0)
    dt_recall = recall_score(y_test, y_pred_dt, average='macro', zero_division=0)
    dt_f1 = f1_score(y_test, y_pred_dt, average='macro', zero_division=0)
    print("\n⚠️  Note: Using multiclass metrics (macro average) because target has more than 2 classes")
else:
    # Binary classification
    dt_accuracy = accuracy_score(y_test, y_pred_dt)
    dt_precision = precision_score(y_test, y_pred_dt, average='binary', zero_division=0)
    dt_recall = recall_score(y_test, y_pred_dt, average='binary', zero_division=0)
    dt_f1 = f1_score(y_test, y_pred_dt, average='binary', zero_division=0)

print("\nDecision Tree Results:")
print(f"Accuracy: {dt_accuracy:.4f}")
print(f"Precision: {dt_precision:.4f}")
print(f"Recall: {dt_recall:.4f}")
print(f"F1-Score: {dt_f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_dt, target_names=['Team 2', 'Team 1']))

In [ ]:
# Feature importance from Decision Tree
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': dt_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(12, 8))
top_features = feature_importance.head(15)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Importance')
plt.title('Top 15 Feature Importances - Decision Tree')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))

In [ ]:
# Confusion Matrix for Decision Tree
cm_dt = confusion_matrix(y_test, y_pred_dt)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_dt, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Team 2', 'Team 1'], 
            yticklabels=['Team 2', 'Team 1'])
plt.title('Confusion Matrix - Decision Tree')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

### Model 3: K-Means Clustering (Unsupervised)

In [ ]:
# K-Means Clustering (using 2 clusters to match binary classification)
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_scaled)

# Evaluate clustering by comparing with actual labels
from sklearn.metrics import adjusted_rand_score, silhouette_score

ari_score = adjusted_rand_score(y_encoded, clusters)
silhouette = silhouette_score(X_scaled, clusters)

print("K-Means Clustering Results:")
print(f"Adjusted Rand Index: {ari_score:.4f}")
print(f"Silhouette Score: {silhouette:.4f}")
print(f"\nCluster distribution: {pd.Series(clusters).value_counts().sort_index()}")
print(f"Actual class distribution: {pd.Series(y_encoded).value_counts().sort_index()}")

In [ ]:
# Visualize clusters using PCA for dimensionality reduction
from sklearn.decomposition import PCA

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(12, 5))

# Plot 1: Actual labels
plt.subplot(1, 2, 1)
scatter1 = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_encoded, cmap='viridis', alpha=0.6)
plt.colorbar(scatter1, label='Actual Class')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title('Actual Classes (PCA Visualization)')

# Plot 2: Predicted clusters
plt.subplot(1, 2, 2)
scatter2 = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='plasma', alpha=0.6)
plt.colorbar(scatter2, label='Cluster')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title('K-Means Clusters (PCA Visualization)')

plt.tight_layout()
plt.show()

print(f"\nPCA Explained Variance Ratio: {pca.explained_variance_ratio_}")
print(f"Total Explained Variance: {sum(pca.explained_variance_ratio_):.4f}")

### Model Comparison

## 1.1.4.1 Model Improvements (Optional Enhancement)

This section explores advanced models and techniques to improve prediction accuracy beyond the baseline models.

### Advanced Model 1: Random Forest

Random Forest combines multiple decision trees to reduce overfitting and improve generalization.

In [ ]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

# Train Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    min_samples_split=20,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)

# Predictions
y_pred_rf = rf_model.predict(X_test)

# Evaluation
if len(np.unique(y_test)) > 2:
    rf_accuracy = accuracy_score(y_test, y_pred_rf)
    rf_precision = precision_score(y_test, y_pred_rf, average='macro', zero_division=0)
    rf_recall = recall_score(y_test, y_pred_rf, average='macro', zero_division=0)
    rf_f1 = f1_score(y_test, y_pred_rf, average='macro', zero_division=0)
else:
    rf_accuracy = accuracy_score(y_test, y_pred_rf)
    rf_precision = precision_score(y_test, y_pred_rf, average='binary', zero_division=0)
    rf_recall = recall_score(y_test, y_pred_rf, average='binary', zero_division=0)
    rf_f1 = f1_score(y_test, y_pred_rf, average='binary', zero_division=0)

print("Random Forest Results:")
print(f"Accuracy: {rf_accuracy:.4f} ({rf_accuracy*100:.2f}%)")
print(f"Precision: {rf_precision:.4f}")
print(f"Recall: {rf_recall:.4f}")
print(f"F1-Score: {rf_f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf, target_names=['Team 2', 'Team 1', 'Other'] if len(np.unique(y_test)) > 2 else ['Team 2', 'Team 1']))

# Feature importance
rf_feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features (Random Forest):")
print(rf_feature_importance.head(10))

### Advanced Model 2: Feature Selection

Selecting the most important features can improve model performance and reduce overfitting.

In [ ]:
# Feature Selection using Random Forest importance
from sklearn.feature_selection import SelectFromModel

# Select top features based on Random Forest importance
selector = SelectFromModel(rf_model, prefit=True, max_features=50)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

selected_features = X_train.columns[selector.get_support()]
print(f"Selected {len(selected_features)} features out of {X_train.shape[1]}")
print(f"\nTop 20 Selected Features:")
for i, feat in enumerate(selected_features[:20], 1):
    print(f"{i}. {feat}")

# Retrain Logistic Regression with selected features
lr_selected = LogisticRegression(random_state=42, max_iter=1000)
lr_selected.fit(X_train_selected, y_train)
y_pred_lr_selected = lr_selected.predict(X_test_selected)

if len(np.unique(y_test)) > 2:
    lr_selected_accuracy = accuracy_score(y_test, y_pred_lr_selected)
    lr_selected_precision = precision_score(y_test, y_pred_lr_selected, average='macro', zero_division=0)
    lr_selected_recall = recall_score(y_test, y_pred_lr_selected, average='macro', zero_division=0)
    lr_selected_f1 = f1_score(y_test, y_pred_lr_selected, average='macro', zero_division=0)
else:
    lr_selected_accuracy = accuracy_score(y_test, y_pred_lr_selected)
    lr_selected_precision = precision_score(y_test, y_pred_lr_selected, average='binary', zero_division=0)
    lr_selected_recall = recall_score(y_test, y_pred_lr_selected, average='binary', zero_division=0)
    lr_selected_f1 = f1_score(y_test, y_pred_lr_selected, average='binary', zero_division=0)

print(f"\nLogistic Regression with Feature Selection:")
print(f"Accuracy: {lr_selected_accuracy:.4f} ({lr_selected_accuracy*100:.2f}%)")
print(f"Precision: {lr_selected_precision:.4f}")
print(f"Recall: {lr_selected_recall:.4f}")
print(f"F1-Score: {lr_selected_f1:.4f}")
print(f"\nImprovement: {((lr_selected_accuracy - lr_accuracy) * 100):.2f}%")

### Advanced Model 3: Hyperparameter Tuning

Using Grid Search to find optimal hyperparameters for Random Forest.

In [ ]:
# Hyperparameter Tuning with Grid Search
from sklearn.model_selection import GridSearchCV

# Define parameter grid (using a smaller grid for faster execution)
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [10, 15, 20],
    'min_samples_split': [10, 20]
}

print("Performing Grid Search (this may take a few minutes)...")
rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    param_grid,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)
rf_grid.fit(X_train, y_train)

print(f"\nBest Parameters: {rf_grid.best_params_}")
print(f"Best Cross-Validation Score: {rf_grid.best_score_:.4f}")

# Evaluate on test set
y_pred_rf_tuned = rf_grid.predict(X_test)

if len(np.unique(y_test)) > 2:
    rf_tuned_accuracy = accuracy_score(y_test, y_pred_rf_tuned)
    rf_tuned_precision = precision_score(y_test, y_pred_rf_tuned, average='macro', zero_division=0)
    rf_tuned_recall = recall_score(y_test, y_pred_rf_tuned, average='macro', zero_division=0)
    rf_tuned_f1 = f1_score(y_test, y_pred_rf_tuned, average='macro', zero_division=0)
else:
    rf_tuned_accuracy = accuracy_score(y_test, y_pred_rf_tuned)
    rf_tuned_precision = precision_score(y_test, y_pred_rf_tuned, average='binary', zero_division=0)
    rf_tuned_recall = recall_score(y_test, y_pred_rf_tuned, average='binary', zero_division=0)
    rf_tuned_f1 = f1_score(y_test, y_pred_rf_tuned, average='binary', zero_division=0)

print(f"\nTuned Random Forest Results:")
print(f"Test Accuracy: {rf_tuned_accuracy:.4f} ({rf_tuned_accuracy*100:.2f}%)")
print(f"Precision: {rf_tuned_precision:.4f}")
print(f"Recall: {rf_tuned_recall:.4f}")
print(f"F1-Score: {rf_tuned_f1:.4f}")
print(f"\nImprovement over baseline RF: {((rf_tuned_accuracy - rf_accuracy) * 100):.2f}%")
print(f"Improvement over Logistic Regression: {((rf_tuned_accuracy - lr_accuracy) * 100):.2f}%")

### Improved Models Comparison

In [ ]:
# Compare all models including improvements
improved_comparison = pd.DataFrame({
    'Model': ['Logistic Regression (Baseline)', 'Decision Tree (Baseline)', 
              'Random Forest', 'LR with Feature Selection', 'Tuned Random Forest'],
    'Accuracy': [lr_accuracy, dt_accuracy, rf_accuracy, 
                 lr_selected_accuracy if 'lr_selected_accuracy' in globals() else np.nan,
                 rf_tuned_accuracy if 'rf_tuned_accuracy' in globals() else np.nan],
    'Precision': [lr_precision, dt_precision, rf_precision,
                  lr_selected_precision if 'lr_selected_precision' in globals() else np.nan,
                  rf_tuned_precision if 'rf_tuned_precision' in globals() else np.nan],
    'Recall': [lr_recall, dt_recall, rf_recall,
               lr_selected_recall if 'lr_selected_recall' in globals() else np.nan,
               rf_tuned_recall if 'rf_tuned_recall' in globals() else np.nan],
    'F1-Score': [lr_f1, dt_f1, rf_f1,
                 lr_selected_f1 if 'lr_selected_f1' in globals() else np.nan,
                 rf_tuned_f1 if 'rf_tuned_f1' in globals() else np.nan]
})

print("="*80)
print("IMPROVED MODELS COMPARISON")
print("="*80)
print(improved_comparison.to_string(index=False))

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Accuracy comparison
models_list = improved_comparison['Model'].tolist()
accuracies = improved_comparison['Accuracy'].tolist()
colors = ['skyblue', 'lightgreen', 'orange', 'purple', 'red']

axes[0].barh(models_list, accuracies, color=colors[:len(models_list)])
axes[0].set_xlabel('Accuracy')
axes[0].set_title('Model Accuracy Comparison (All Models)')
axes[0].set_xlim([0.45, max([a for a in accuracies if not np.isnan(a)]) * 1.1])
axes[0].grid(axis='x', alpha=0.3)

# Add value labels
for i, acc in enumerate(accuracies):
    if not np.isnan(acc):
        axes[0].text(acc + 0.005, i, f'{acc:.3f}', va='center')

# Metrics comparison for top 3 models
top_models = improved_comparison.nlargest(3, 'Accuracy')
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
x = np.arange(len(metrics))
width = 0.25

for i, (idx, row) in enumerate(top_models.iterrows()):
    model_metrics = [row['Accuracy'], row['Precision'], row['Recall'], row['F1-Score']]
    axes[1].bar(x + i*width, model_metrics, width, label=row['Model'][:20], alpha=0.8)

axes[1].set_ylabel('Score')
axes[1].set_title('Top 3 Models - Detailed Metrics')
axes[1].set_xticks(x + width)
axes[1].set_xticklabels(metrics)
axes[1].legend()
axes[1].set_ylim([0, 1])
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Find best model
best_model_idx = improved_comparison['Accuracy'].idxmax()
best_model_name = improved_comparison.loc[best_model_idx, 'Model']
best_model_acc = improved_comparison.loc[best_model_idx, 'Accuracy']

print(f"\n🏆 Best Model: {best_model_name}")
print(f"   Accuracy: {best_model_acc:.4f} ({best_model_acc*100:.2f}%)")
print(f"   Improvement over baseline LR: {((best_model_acc - lr_accuracy) * 100):.2f}%")

### Additional Feature Engineering Ideas

Here are some additional features that could potentially improve model performance:

In [ ]:
# Additional Feature Engineering (Optional - for experimentation)
# These features could be added to potentially improve performance

df_enhanced = df_features.copy()

# 1. Team strength difference (using average ratings)
if 't1_avg_rating' in df_enhanced.columns and 't2_avg_rating' in df_enhanced.columns:
    df_enhanced['rating_diff'] = df_enhanced['t1_avg_rating'] - df_enhanced['t2_avg_rating']

# 2. Impact difference
if 't1_avg_impact' in df_enhanced.columns and 't2_avg_impact' in df_enhanced.columns:
    df_enhanced['impact_diff'] = df_enhanced['t1_avg_impact'] - df_enhanced['t2_avg_impact']

# 3. KDR difference
if 't1_avg_kdr' in df_enhanced.columns and 't2_avg_kdr' in df_enhanced.columns:
    df_enhanced['kdr_diff'] = df_enhanced['t1_avg_kdr'] - df_enhanced['t2_avg_kdr']

# 4. Team consistency (standard deviation of player ratings)
t1_rating_cols = [col for col in df_enhanced.columns if 't1_player' in col and 'rating' in col]
t2_rating_cols = [col for col in df_enhanced.columns if 't2_player' in col and 'rating' in col]

if len(t1_rating_cols) > 0 and len(t2_rating_cols) > 0:
    df_enhanced['t1_rating_std'] = df_enhanced[t1_rating_cols].std(axis=1)
    df_enhanced['t2_rating_std'] = df_enhanced[t2_rating_cols].std(axis=1)
    df_enhanced['rating_consistency_diff'] = df_enhanced['t1_rating_std'] - df_enhanced['t2_rating_std']

# 5. Sniper advantage (teams with more snipers)
t1_sniper_cols = [col for col in df_enhanced.columns if 't1_player' in col and 'is_sniper' in col]
t2_sniper_cols = [col for col in df_enhanced.columns if 't2_player' in col and 'is_sniper' in col]

if len(t1_sniper_cols) > 0 and len(t2_sniper_cols) > 0:
    df_enhanced['t1_sniper_count'] = df_enhanced[t1_sniper_cols].sum(axis=1)
    df_enhanced['t2_sniper_count'] = df_enhanced[t2_sniper_cols].sum(axis=1)
    df_enhanced['sniper_diff'] = df_enhanced['t1_sniper_count'] - df_enhanced['t2_sniper_count']

print("Additional features created:")
new_enhanced_features = [col for col in df_enhanced.columns if col not in df_features.columns]
print(new_enhanced_features)
print(f"\nTotal new features: {len(new_enhanced_features)}")

# Note: To use these features, you would need to:
# 1. Re-prepare X with these new features
# 2. Re-scale
# 3. Re-train models
# This is left as an exercise for further experimentation

## Advanced Model Improvements - Attempting Higher Accuracy

We'll now try more advanced techniques to push accuracy beyond 57.77%:
1. **XGBoost** - State-of-the-art gradient boosting
2. **Gradient Boosting** - Alternative boosting method
3. **Voting Ensemble** - Combine multiple models
4. **Extended Hyperparameter Tuning** - More thorough search
5. **Polynomial Features** - Capture feature interactions

In [ ]:
# Advanced Model 4: XGBoost (if available)
# XGBoost is often the best-performing model for structured data
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
    print("✓ XGBoost is available")
except ImportError:
    XGBOOST_AVAILABLE = False
    print("⚠ XGBoost not available. Install with: !pip install xgboost")
    print("   Continuing with other models...")

if XGBOOST_AVAILABLE:
    print("\n" + "="*80)
    print("Training XGBoost Classifier...")
    print("="*80)
    
    # XGBoost with default parameters
    xgb_model = xgb.XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        eval_metric='logloss'
    )
    
    xgb_model.fit(X_train, y_train)
    y_pred_xgb = xgb_model.predict(X_test)
    
    if len(np.unique(y_test)) > 2:
        xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
        xgb_precision = precision_score(y_test, y_pred_xgb, average='macro', zero_division=0)
        xgb_recall = recall_score(y_test, y_pred_xgb, average='macro', zero_division=0)
        xgb_f1 = f1_score(y_test, y_pred_xgb, average='macro', zero_division=0)
    else:
        xgb_accuracy = accuracy_score(y_test, y_pred_xgb)
        xgb_precision = precision_score(y_test, y_pred_xgb, average='binary', zero_division=0)
        xgb_recall = recall_score(y_test, y_pred_xgb, average='binary', zero_division=0)
        xgb_f1 = f1_score(y_test, y_pred_xgb, average='binary', zero_division=0)
    
    print(f"\nXGBoost Results:")
    print(f"Accuracy: {xgb_accuracy:.4f} ({xgb_accuracy*100:.2f}%)")
    print(f"Precision: {xgb_precision:.4f}")
    print(f"Recall: {xgb_recall:.4f}")
    print(f"F1-Score: {xgb_f1:.4f}")
    print(f"\nImprovement over baseline LR: {((xgb_accuracy - lr_accuracy) * 100):.2f}%")
    if 'rf_tuned_accuracy' in globals():
        print(f"Improvement over Tuned RF: {((xgb_accuracy - rf_tuned_accuracy) * 100):.2f}%")
    
    # Feature importance
    feature_importance_xgb = pd.DataFrame({
        'feature': X.columns,
        'importance': xgb_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\nTop 10 Most Important Features (XGBoost):")
    print(feature_importance_xgb.head(10).to_string(index=False))
else:
    print("\nSkipping XGBoost (not available)")

In [ ]:
# Advanced Model 5: Gradient Boosting (scikit-learn)
from sklearn.ensemble import GradientBoostingClassifier

print("\n" + "="*80)
print("Training Gradient Boosting Classifier...")
print("="*80)

gb_model = GradientBoostingClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    subsample=0.8,
    random_state=42
)

gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)

if len(np.unique(y_test)) > 2:
    gb_accuracy = accuracy_score(y_test, y_pred_gb)
    gb_precision = precision_score(y_test, y_pred_gb, average='macro', zero_division=0)
    gb_recall = recall_score(y_test, y_pred_gb, average='macro', zero_division=0)
    gb_f1 = f1_score(y_test, y_pred_gb, average='macro', zero_division=0)
else:
    gb_accuracy = accuracy_score(y_test, y_pred_gb)
    gb_precision = precision_score(y_test, y_pred_gb, average='binary', zero_division=0)
    gb_recall = recall_score(y_test, y_pred_gb, average='binary', zero_division=0)
    gb_f1 = f1_score(y_test, y_pred_gb, average='binary', zero_division=0)

print(f"\nGradient Boosting Results:")
print(f"Accuracy: {gb_accuracy:.4f} ({gb_accuracy*100:.2f}%)")
print(f"Precision: {gb_precision:.4f}")
print(f"Recall: {gb_recall:.4f}")
print(f"F1-Score: {gb_f1:.4f}")
print(f"\nImprovement over baseline LR: {((gb_accuracy - lr_accuracy) * 100):.2f}%")
if 'rf_tuned_accuracy' in globals():
    print(f"Improvement over Tuned RF: {((gb_accuracy - rf_tuned_accuracy) * 100):.2f}%")

In [ ]:
# Advanced Model 6: Voting Ensemble (combine multiple models)
from sklearn.ensemble import VotingClassifier

print("\n" + "="*80)
print("Training Voting Ensemble Classifier...")
print("="*80)

# Create a voting classifier with our best models
estimators = [
    ('lr', LogisticRegression(random_state=42, max_iter=1000)),
    ('rf', RandomForestClassifier(n_estimators=100, max_depth=15, min_samples_split=20, random_state=42, n_jobs=-1)),
    ('gb', GradientBoostingClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42))
]

# Add XGBoost if available
if XGBOOST_AVAILABLE:
    estimators.append(('xgb', xgb.XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42, n_jobs=-1, eval_metric='logloss')))

voting_model = VotingClassifier(estimators=estimators, voting='soft')
voting_model.fit(X_train, y_train)
y_pred_voting = voting_model.predict(X_test)

if len(np.unique(y_test)) > 2:
    voting_accuracy = accuracy_score(y_test, y_pred_voting)
    voting_precision = precision_score(y_test, y_pred_voting, average='macro', zero_division=0)
    voting_recall = recall_score(y_test, y_pred_voting, average='macro', zero_division=0)
    voting_f1 = f1_score(y_test, y_pred_voting, average='macro', zero_division=0)
else:
    voting_accuracy = accuracy_score(y_test, y_pred_voting)
    voting_precision = precision_score(y_test, y_pred_voting, average='binary', zero_division=0)
    voting_recall = recall_score(y_test, y_pred_voting, average='binary', zero_division=0)
    voting_f1 = f1_score(y_test, y_pred_voting, average='binary', zero_division=0)

print(f"\nVoting Ensemble Results:")
print(f"Accuracy: {voting_accuracy:.4f} ({voting_accuracy*100:.2f}%)")
print(f"Precision: {voting_precision:.4f}")
print(f"Recall: {voting_recall:.4f}")
print(f"F1-Score: {voting_f1:.4f}")
print(f"\nImprovement over baseline LR: {((voting_accuracy - lr_accuracy) * 100):.2f}%")
if 'rf_tuned_accuracy' in globals():
    print(f"Improvement over Tuned RF: {((voting_accuracy - rf_tuned_accuracy) * 100):.2f}%")

In [ ]:
# Advanced Model 7: Extended Hyperparameter Tuning for XGBoost (if available)
if XGBOOST_AVAILABLE:
    print("\n" + "="*80)
    print("Extended Hyperparameter Tuning for XGBoost...")
    print("="*80)
    
    # More extensive parameter grid for XGBoost
    xgb_param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [4, 6, 8],
        'learning_rate': [0.05, 0.1],
        'subsample': [0.8, 0.9]
    }
    
    print("Performing Grid Search for XGBoost (this may take several minutes)...")
    xgb_grid = GridSearchCV(
        xgb.XGBClassifier(random_state=42, n_jobs=-1, eval_metric='logloss'),
        xgb_param_grid,
        cv=3,
        scoring='accuracy',
        n_jobs=-1,
        verbose=1
    )
    
    xgb_grid.fit(X_train, y_train)
    
    print(f"\nBest Parameters: {xgb_grid.best_params_}")
    print(f"Best Cross-Validation Score: {xgb_grid.best_score_:.4f}")
    
    # Evaluate on test set
    y_pred_xgb_tuned = xgb_grid.predict(X_test)
    
    if len(np.unique(y_test)) > 2:
        xgb_tuned_accuracy = accuracy_score(y_test, y_pred_xgb_tuned)
        xgb_tuned_precision = precision_score(y_test, y_pred_xgb_tuned, average='macro', zero_division=0)
        xgb_tuned_recall = recall_score(y_test, y_pred_xgb_tuned, average='macro', zero_division=0)
        xgb_tuned_f1 = f1_score(y_test, y_pred_xgb_tuned, average='macro', zero_division=0)
    else:
        xgb_tuned_accuracy = accuracy_score(y_test, y_pred_xgb_tuned)
        xgb_tuned_precision = precision_score(y_test, y_pred_xgb_tuned, average='binary', zero_division=0)
        xgb_tuned_recall = recall_score(y_test, y_pred_xgb_tuned, average='binary', zero_division=0)
        xgb_tuned_f1 = f1_score(y_test, y_pred_xgb_tuned, average='binary', zero_division=0)
    
    print(f"\nTuned XGBoost Results:")
    print(f"Test Accuracy: {xgb_tuned_accuracy:.4f} ({xgb_tuned_accuracy*100:.2f}%)")
    print(f"Precision: {xgb_tuned_precision:.4f}")
    print(f"Recall: {xgb_tuned_recall:.4f}")
    print(f"F1-Score: {xgb_tuned_f1:.4f}")
    print(f"\nImprovement over baseline LR: {((xgb_tuned_accuracy - lr_accuracy) * 100):.2f}%")
    if 'rf_tuned_accuracy' in globals():
        print(f"Improvement over Tuned RF: {((xgb_tuned_accuracy - rf_tuned_accuracy) * 100):.2f}%")
else:
    print("\nSkipping XGBoost tuning (XGBoost not available)")

In [17]:
# Final Comparison of ALL Models
print("\n" + "="*80)
print("FINAL COMPREHENSIVE MODEL COMPARISON")
print("="*80)

# Collect all model results
all_model_results = {
    'Model': ['Logistic Regression', 'Decision Tree', 'Random Forest', 
              'Tuned Random Forest', 'Gradient Boosting'],
    'Accuracy': [lr_accuracy, dt_accuracy, rf_accuracy, 
                 rf_tuned_accuracy if 'rf_tuned_accuracy' in globals() else np.nan,
                 gb_accuracy if 'gb_accuracy' in globals() else np.nan],
    'Precision': [lr_precision, dt_precision, rf_precision,
                  rf_tuned_precision if 'rf_tuned_precision' in globals() else np.nan,
                  gb_precision if 'gb_precision' in globals() else np.nan],
    'Recall': [lr_recall, dt_recall, rf_recall,
               rf_tuned_recall if 'rf_tuned_recall' in globals() else np.nan,
               gb_recall if 'gb_recall' in globals() else np.nan],
    'F1-Score': [lr_f1, dt_f1, rf_f1,
                 rf_tuned_f1 if 'rf_tuned_f1' in globals() else np.nan,
                 gb_f1 if 'gb_f1' in globals() else np.nan]
}

# Add XGBoost if available
if XGBOOST_AVAILABLE and 'xgb_accuracy' in globals():
    all_model_results['Model'].append('XGBoost')
    all_model_results['Accuracy'].append(xgb_accuracy)
    all_model_results['Precision'].append(xgb_precision)
    all_model_results['Recall'].append(xgb_recall)
    all_model_results['F1-Score'].append(xgb_f1)
    
    if 'xgb_tuned_accuracy' in globals():
        all_model_results['Model'].append('Tuned XGBoost')
        all_model_results['Accuracy'].append(xgb_tuned_accuracy)
        all_model_results['Precision'].append(xgb_tuned_precision)
        all_model_results['Recall'].append(xgb_tuned_recall)
        all_model_results['F1-Score'].append(xgb_tuned_f1)

# Add Voting Ensemble
if 'voting_accuracy' in globals():
    all_model_results['Model'].append('Voting Ensemble')
    all_model_results['Accuracy'].append(voting_accuracy)
    all_model_results['Precision'].append(voting_precision)
    all_model_results['Recall'].append(voting_recall)
    all_model_results['F1-Score'].append(voting_f1)

final_comparison = pd.DataFrame(all_model_results)
final_comparison = final_comparison.sort_values('Accuracy', ascending=False)

print(final_comparison.to_string(index=False))

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Accuracy comparison
models_list = final_comparison['Model'].tolist()
accuracies = final_comparison['Accuracy'].tolist()
colors = plt.cm.viridis(np.linspace(0, 1, len(models_list)))

axes[0].barh(models_list, accuracies, color=colors)
axes[0].set_xlabel('Accuracy', fontsize=12)
axes[0].set_title('All Models - Accuracy Comparison', fontsize=14, fontweight='bold')
axes[0].set_xlim([0.45, max([a for a in accuracies if not np.isnan(a)]) * 1.1])
axes[0].grid(axis='x', alpha=0.3)

# Add value labels
for i, acc in enumerate(accuracies):
    if not np.isnan(acc):
        axes[0].text(acc + 0.005, i, f'{acc:.4f} ({acc*100:.2f}%)', va='center', fontsize=10)

# Metrics comparison for top 5 models
top_models = final_comparison.nlargest(5, 'Accuracy')
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
x = np.arange(len(metrics))
width = 0.15

for i, (idx, row) in enumerate(top_models.iterrows()):
    model_metrics = [row['Accuracy'], row['Precision'], row['Recall'], row['F1-Score']]
    axes[1].bar(x + i*width, model_metrics, width, label=row['Model'][:25], alpha=0.8)

axes[1].set_ylabel('Score', fontsize=12)
axes[1].set_title('Top 5 Models - Detailed Metrics', fontsize=14, fontweight='bold')
axes[1].set_xticks(x + width * 2)
axes[1].set_xticklabels(metrics)
axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1].set_ylim([0, 1])
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Find best model
best_model_idx = final_comparison['Accuracy'].idxmax()
best_model_name = final_comparison.loc[best_model_idx, 'Model']
best_model_acc = final_comparison.loc[best_model_idx, 'Accuracy']

print(f"\n{'='*80}")
print(f"🏆 BEST MODEL: {best_model_name}")
print(f"{'='*80}")
print(f"   Accuracy: {best_model_acc:.4f} ({best_model_acc*100:.2f}%)")
print(f"   Precision: {final_comparison.loc[best_model_idx, 'Precision']:.4f}")
print(f"   Recall: {final_comparison.loc[best_model_idx, 'Recall']:.4f}")
print(f"   F1-Score: {final_comparison.loc[best_model_idx, 'F1-Score']:.4f}")
print(f"\n   Improvement over baseline LR: {((best_model_acc - lr_accuracy) * 100):.2f}%")
if 'rf_tuned_accuracy' in globals():
    print(f"   Improvement over previous best (Tuned RF): {((best_model_acc - rf_tuned_accuracy) * 100):.2f}%")
print(f"{'='*80}")


FINAL COMPREHENSIVE MODEL COMPARISON


NameError: name 'lr_accuracy' is not defined

## Pushing Beyond 60% - Advanced Techniques

We'll now try even more sophisticated techniques to push accuracy above 60%:
1. **Stacking Ensemble** - Use model predictions as features
2. **Polynomial Features** - Capture feature interactions
3. **LightGBM** - Alternative gradient boosting (often faster and sometimes better)
4. **Extended Feature Engineering** - More sophisticated feature creation
5. **Advanced Hyperparameter Tuning** - Larger search spaces

In [ ]:
# Advanced Technique 1: Stacking Ensemble
# Use predictions from base models as features for a meta-model
from sklearn.model_selection import cross_val_predict

print("\n" + "="*80)
print("Training Stacking Ensemble...")
print("="*80)

# Create base models
base_models = [
    ('lr', LogisticRegression(random_state=42, max_iter=1000)),
    ('rf', RandomForestClassifier(n_estimators=100, max_depth=15, min_samples_split=20, random_state=42, n_jobs=-1)),
    ('gb', GradientBoostingClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42))
]

# Add XGBoost if available
if XGBOOST_AVAILABLE and 'xgb_model' in globals():
    base_models.append(('xgb', xgb_model))

# Generate meta-features using cross-validation (to avoid overfitting)
print("Generating meta-features using cross-validation...")
meta_features_train = np.zeros((X_train.shape[0], len(base_models)))
meta_features_test = np.zeros((X_test.shape[0], len(base_models)))

for i, (name, model) in enumerate(base_models):
    print(f"  Training {name} for meta-features...")
    # Use cross-validation to get out-of-fold predictions
    meta_features_train[:, i] = cross_val_predict(model, X_train, y_train, cv=5, method='predict_proba')[:, 1]
    # Train on full training set and predict on test
    model.fit(X_train, y_train)
    meta_features_test[:, i] = model.predict_proba(X_test)[:, 1]

# Train meta-model on meta-features
from sklearn.linear_model import LogisticRegression as MetaLR
meta_model = MetaLR(random_state=42, max_iter=1000)
meta_model.fit(meta_features_train, y_train)
y_pred_stacking = meta_model.predict(meta_features_test)

if len(np.unique(y_test)) > 2:
    stacking_accuracy = accuracy_score(y_test, y_pred_stacking)
    stacking_precision = precision_score(y_test, y_pred_stacking, average='macro', zero_division=0)
    stacking_recall = recall_score(y_test, y_pred_stacking, average='macro', zero_division=0)
    stacking_f1 = f1_score(y_test, y_pred_stacking, average='macro', zero_division=0)
else:
    stacking_accuracy = accuracy_score(y_test, y_pred_stacking)
    stacking_precision = precision_score(y_test, y_pred_stacking, average='binary', zero_division=0)
    stacking_recall = recall_score(y_test, y_pred_stacking, average='binary', zero_division=0)
    stacking_f1 = f1_score(y_test, y_pred_stacking, average='binary', zero_division=0)

print(f"\nStacking Ensemble Results:")
print(f"Accuracy: {stacking_accuracy:.4f} ({stacking_accuracy*100:.2f}%)")
print(f"Precision: {stacking_precision:.4f}")
print(f"Recall: {stacking_recall:.4f}")
print(f"F1-Score: {stacking_f1:.4f}")
if 'xgb_tuned_accuracy' in globals():
    print(f"\nImprovement over Tuned XGBoost: {((stacking_accuracy - xgb_tuned_accuracy) * 100):.2f}%")
elif 'rf_tuned_accuracy' in globals():
    print(f"\nImprovement over Tuned RF: {((stacking_accuracy - rf_tuned_accuracy) * 100):.2f}%")

In [ ]:
# Advanced Technique 2: LightGBM (if available)
# LightGBM is often faster and sometimes more accurate than XGBoost
try:
    import lightgbm as lgb
    LIGHTGBM_AVAILABLE = True
    print("✓ LightGBM is available")
except ImportError:
    LIGHTGBM_AVAILABLE = False
    print("⚠ LightGBM not available. Install with: !pip install lightgbm")
    print("   Continuing with other models...")

if LIGHTGBM_AVAILABLE:
    print("\n" + "="*80)
    print("Training LightGBM Classifier...")
    print("="*80)
    
    # LightGBM with optimized parameters
    lgb_model = lgb.LGBMClassifier(
        n_estimators=200,
        max_depth=7,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    )
    
    lgb_model.fit(X_train, y_train)
    y_pred_lgb = lgb_model.predict(X_test)
    
    if len(np.unique(y_test)) > 2:
        lgb_accuracy = accuracy_score(y_test, y_pred_lgb)
        lgb_precision = precision_score(y_test, y_pred_lgb, average='macro', zero_division=0)
        lgb_recall = recall_score(y_test, y_pred_lgb, average='macro', zero_division=0)
        lgb_f1 = f1_score(y_test, y_pred_lgb, average='macro', zero_division=0)
    else:
        lgb_accuracy = accuracy_score(y_test, y_pred_lgb)
        lgb_precision = precision_score(y_test, y_pred_lgb, average='binary', zero_division=0)
        lgb_recall = recall_score(y_test, y_pred_lgb, average='binary', zero_division=0)
        lgb_f1 = f1_score(y_test, y_pred_lgb, average='binary', zero_division=0)
    
    print(f"\nLightGBM Results:")
    print(f"Accuracy: {lgb_accuracy:.4f} ({lgb_accuracy*100:.2f}%)")
    print(f"Precision: {lgb_precision:.4f}")
    print(f"Recall: {lgb_recall:.4f}")
    print(f"F1-Score: {lgb_f1:.4f}")
    if 'xgb_tuned_accuracy' in globals():
        print(f"\nImprovement over Tuned XGBoost: {((lgb_accuracy - xgb_tuned_accuracy) * 100):.2f}%")
    elif 'rf_tuned_accuracy' in globals():
        print(f"\nImprovement over Tuned RF: {((lgb_accuracy - rf_tuned_accuracy) * 100):.2f}%")
    
    # Feature importance
    feature_importance_lgb = pd.DataFrame({
        'feature': X.columns,
        'importance': lgb_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\nTop 10 Most Important Features (LightGBM):")
    print(feature_importance_lgb.head(10).to_string(index=False))
else:
    print("\nSkipping LightGBM (not available)")

In [ ]:
# Advanced Technique 3: Polynomial Features (for key interactions)
# Create polynomial features for the most important features
from sklearn.preprocessing import PolynomialFeatures

print("\n" + "="*80)
print("Creating Polynomial Features for Top Features...")
print("="*80)

# Get top features from XGBoost if available, otherwise from Random Forest
if XGBOOST_AVAILABLE and 'xgb_model' in globals():
    top_features_idx = np.argsort(xgb_model.feature_importances_)[-20:]  # Top 20 features
    top_feature_names = X.columns[top_features_idx].tolist()
elif 'rf_model' in globals():
    top_features_idx = np.argsort(rf_model.feature_importances_)[-20:]
    top_feature_names = X.columns[top_features_idx].tolist()
else:
    # Use correlation-based selection
    correlations = X_train.corrwith(pd.Series(y_train, index=X_train.index)).abs()
    top_feature_names = correlations.nlargest(20).index.tolist()

print(f"Selected top {len(top_feature_names)} features for polynomial expansion")

# Create polynomial features (degree 2) for top features only
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_train_top = X_train[top_feature_names]
X_test_top = X_test[top_feature_names]

X_train_poly = poly.fit_transform(X_train_top)
X_test_poly = poly.transform(X_test_top)

print(f"Original features: {X_train_top.shape[1]}")
print(f"Polynomial features: {X_train_poly.shape[1]}")

# Combine original features with polynomial features
from scipy.sparse import hstack
from scipy import sparse

# Convert to sparse if needed for memory efficiency
X_train_combined = np.hstack([X_train.values, X_train_poly])
X_test_combined = np.hstack([X_test.values, X_test_poly])

print(f"Combined features: {X_train_combined.shape[1]}")

# Train XGBoost on combined features (if available)
if XGBOOST_AVAILABLE:
    print("\nTraining XGBoost on combined features...")
    xgb_poly = xgb.XGBClassifier(
        n_estimators=150,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        eval_metric='logloss'
    )
    
    xgb_poly.fit(X_train_combined, y_train)
    y_pred_xgb_poly = xgb_poly.predict(X_test_combined)
    
    if len(np.unique(y_test)) > 2:
        xgb_poly_accuracy = accuracy_score(y_test, y_pred_xgb_poly)
        xgb_poly_precision = precision_score(y_test, y_pred_xgb_poly, average='macro', zero_division=0)
        xgb_poly_recall = recall_score(y_test, y_pred_xgb_poly, average='macro', zero_division=0)
        xgb_poly_f1 = f1_score(y_test, y_pred_xgb_poly, average='macro', zero_division=0)
    else:
        xgb_poly_accuracy = accuracy_score(y_test, y_pred_xgb_poly)
        xgb_poly_precision = precision_score(y_test, y_pred_xgb_poly, average='binary', zero_division=0)
        xgb_poly_recall = recall_score(y_test, y_pred_xgb_poly, average='binary', zero_division=0)
        xgb_poly_f1 = f1_score(y_test, y_pred_xgb_poly, average='binary', zero_division=0)
    
    print(f"\nXGBoost with Polynomial Features Results:")
    print(f"Accuracy: {xgb_poly_accuracy:.4f} ({xgb_poly_accuracy*100:.2f}%)")
    print(f"Precision: {xgb_poly_precision:.4f}")
    print(f"Recall: {xgb_poly_recall:.4f}")
    print(f"F1-Score: {xgb_poly_f1:.4f}")
    if 'xgb_tuned_accuracy' in globals():
        print(f"\nImprovement over Tuned XGBoost: {((xgb_poly_accuracy - xgb_tuned_accuracy) * 100):.2f}%")
else:
    print("\nSkipping polynomial XGBoost (XGBoost not available)")

In [ ]:
# Advanced Technique 4: Extended Hyperparameter Tuning for LightGBM (if available)
if LIGHTGBM_AVAILABLE:
    print("\n" + "="*80)
    print("Extended Hyperparameter Tuning for LightGBM...")
    print("="*80)
    
    # More extensive parameter grid for LightGBM
    lgb_param_grid = {
        'n_estimators': [200, 300],
        'max_depth': [5, 7, 9],
        'learning_rate': [0.03, 0.05, 0.1],
        'subsample': [0.8, 0.9]
    }
    
    print("Performing Grid Search for LightGBM (this may take several minutes)...")
    lgb_grid = GridSearchCV(
        lgb.LGBMClassifier(random_state=42, n_jobs=-1, verbose=-1),
        lgb_param_grid,
        cv=3,
        scoring='accuracy',
        n_jobs=-1,
        verbose=1
    )
    
    lgb_grid.fit(X_train, y_train)
    
    print(f"\nBest Parameters: {lgb_grid.best_params_}")
    print(f"Best Cross-Validation Score: {lgb_grid.best_score_:.4f}")
    
    # Evaluate on test set
    y_pred_lgb_tuned = lgb_grid.predict(X_test)
    
    if len(np.unique(y_test)) > 2:
        lgb_tuned_accuracy = accuracy_score(y_test, y_pred_lgb_tuned)
        lgb_tuned_precision = precision_score(y_test, y_pred_lgb_tuned, average='macro', zero_division=0)
        lgb_tuned_recall = recall_score(y_test, y_pred_lgb_tuned, average='macro', zero_division=0)
        lgb_tuned_f1 = f1_score(y_test, y_pred_lgb_tuned, average='macro', zero_division=0)
    else:
        lgb_tuned_accuracy = accuracy_score(y_test, y_pred_lgb_tuned)
        lgb_tuned_precision = precision_score(y_test, y_pred_lgb_tuned, average='binary', zero_division=0)
        lgb_tuned_recall = recall_score(y_test, y_pred_lgb_tuned, average='binary', zero_division=0)
        lgb_tuned_f1 = f1_score(y_test, y_pred_lgb_tuned, average='binary', zero_division=0)
    
    print(f"\nTuned LightGBM Results:")
    print(f"Test Accuracy: {lgb_tuned_accuracy:.4f} ({lgb_tuned_accuracy*100:.2f}%)")
    print(f"Precision: {lgb_tuned_precision:.4f}")
    print(f"Recall: {lgb_tuned_recall:.4f}")
    print(f"F1-Score: {lgb_tuned_f1:.4f}")
    if 'xgb_tuned_accuracy' in globals():
        print(f"\nImprovement over Tuned XGBoost: {((lgb_tuned_accuracy - xgb_tuned_accuracy) * 100):.2f}%")
else:
    print("\nSkipping LightGBM tuning (LightGBM not available)")

In [ ]:
# Final Comprehensive Comparison - ALL Advanced Models
print("\n" + "="*80)
print("FINAL COMPREHENSIVE MODEL COMPARISON (ALL TECHNIQUES)")
print("="*80)

# Collect all model results including new advanced models
all_advanced_results = {
    'Model': ['Logistic Regression', 'Decision Tree', 'Random Forest', 
              'Tuned Random Forest', 'Gradient Boosting'],
    'Accuracy': [lr_accuracy, dt_accuracy, rf_accuracy, 
                 rf_tuned_accuracy if 'rf_tuned_accuracy' in globals() else np.nan,
                 gb_accuracy if 'gb_accuracy' in globals() else np.nan],
    'Precision': [lr_precision, dt_precision, rf_precision,
                  rf_tuned_precision if 'rf_tuned_precision' in globals() else np.nan,
                  gb_precision if 'gb_precision' in globals() else np.nan],
    'Recall': [lr_recall, dt_recall, rf_recall,
               rf_tuned_recall if 'rf_tuned_recall' in globals() else np.nan,
               gb_recall if 'gb_recall' in globals() else np.nan],
    'F1-Score': [lr_f1, dt_f1, rf_f1,
                 rf_tuned_f1 if 'rf_tuned_f1' in globals() else np.nan,
                 gb_f1 if 'gb_f1' in globals() else np.nan]
}

# Add XGBoost models
if XGBOOST_AVAILABLE and 'xgb_accuracy' in globals():
    all_advanced_results['Model'].append('XGBoost')
    all_advanced_results['Accuracy'].append(xgb_accuracy)
    all_advanced_results['Precision'].append(xgb_precision)
    all_advanced_results['Recall'].append(xgb_recall)
    all_advanced_results['F1-Score'].append(xgb_f1)
    
    if 'xgb_tuned_accuracy' in globals():
        all_advanced_results['Model'].append('Tuned XGBoost')
        all_advanced_results['Accuracy'].append(xgb_tuned_accuracy)
        all_advanced_results['Precision'].append(xgb_tuned_precision)
        all_advanced_results['Recall'].append(xgb_tuned_recall)
        all_advanced_results['F1-Score'].append(xgb_tuned_f1)
    
    if 'xgb_poly_accuracy' in globals():
        all_advanced_results['Model'].append('XGBoost + Polynomial')
        all_advanced_results['Accuracy'].append(xgb_poly_accuracy)
        all_advanced_results['Precision'].append(xgb_poly_precision)
        all_advanced_results['Recall'].append(xgb_poly_recall)
        all_advanced_results['F1-Score'].append(xgb_poly_f1)

# Add LightGBM models
if LIGHTGBM_AVAILABLE and 'lgb_accuracy' in globals():
    all_advanced_results['Model'].append('LightGBM')
    all_advanced_results['Accuracy'].append(lgb_accuracy)
    all_advanced_results['Precision'].append(lgb_precision)
    all_advanced_results['Recall'].append(lgb_recall)
    all_advanced_results['F1-Score'].append(lgb_f1)
    
    if 'lgb_tuned_accuracy' in globals():
        all_advanced_results['Model'].append('Tuned LightGBM')
        all_advanced_results['Accuracy'].append(lgb_tuned_accuracy)
        all_advanced_results['Precision'].append(lgb_tuned_precision)
        all_advanced_results['Recall'].append(lgb_tuned_recall)
        all_advanced_results['F1-Score'].append(lgb_tuned_f1)

# Add ensemble models
if 'voting_accuracy' in globals():
    all_advanced_results['Model'].append('Voting Ensemble')
    all_advanced_results['Accuracy'].append(voting_accuracy)
    all_advanced_results['Precision'].append(voting_precision)
    all_advanced_results['Recall'].append(voting_recall)
    all_advanced_results['F1-Score'].append(voting_f1)

if 'stacking_accuracy' in globals():
    all_advanced_results['Model'].append('Stacking Ensemble')
    all_advanced_results['Accuracy'].append(stacking_accuracy)
    all_advanced_results['Precision'].append(stacking_precision)
    all_advanced_results['Recall'].append(stacking_recall)
    all_advanced_results['F1-Score'].append(stacking_f1)

final_advanced_comparison = pd.DataFrame(all_advanced_results)
final_advanced_comparison = final_advanced_comparison.sort_values('Accuracy', ascending=False)

print(final_advanced_comparison.to_string(index=False))

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Accuracy comparison
models_list = final_advanced_comparison['Model'].tolist()
accuracies = final_advanced_comparison['Accuracy'].tolist()
colors = plt.cm.plasma(np.linspace(0, 1, len(models_list)))

axes[0].barh(models_list, accuracies, color=colors)
axes[0].set_xlabel('Accuracy', fontsize=12)
axes[0].set_title('All Models - Accuracy Comparison (Including Advanced Techniques)', fontsize=14, fontweight='bold')
axes[0].set_xlim([0.45, max([a for a in accuracies if not np.isnan(a)]) * 1.15])
axes[0].grid(axis='x', alpha=0.3)
axes[0].axvline(x=0.60, color='red', linestyle='--', linewidth=2, label='60% Target')
axes[0].legend()

# Add value labels
for i, acc in enumerate(accuracies):
    if not np.isnan(acc):
        axes[0].text(acc + 0.005, i, f'{acc:.4f} ({acc*100:.2f}%)', va='center', fontsize=9)

# Metrics comparison for top 5 models
top_models = final_advanced_comparison.nlargest(5, 'Accuracy')
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
x = np.arange(len(metrics))
width = 0.15

for i, (idx, row) in enumerate(top_models.iterrows()):
    model_metrics = [row['Accuracy'], row['Precision'], row['Recall'], row['F1-Score']]
    axes[1].bar(x + i*width, model_metrics, width, label=row['Model'][:25], alpha=0.8)

axes[1].set_ylabel('Score', fontsize=12)
axes[1].set_title('Top 5 Models - Detailed Metrics', fontsize=14, fontweight='bold')
axes[1].set_xticks(x + width * 2)
axes[1].set_xticklabels(metrics)
axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1].set_ylim([0, 1])
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Find best model
best_model_idx = final_advanced_comparison['Accuracy'].idxmax()
best_model_name = final_advanced_comparison.loc[best_model_idx, 'Model']
best_model_acc = final_advanced_comparison.loc[best_model_idx, 'Accuracy']

print(f"\n{'='*80}")
print(f"🏆 BEST MODEL: {best_model_name}")
print(f"{'='*80}")
print(f"   Accuracy: {best_model_acc:.4f} ({best_model_acc*100:.2f}%)")
print(f"   Precision: {final_advanced_comparison.loc[best_model_idx, 'Precision']:.4f}")
print(f"   Recall: {final_advanced_comparison.loc[best_model_idx, 'Recall']:.4f}")
print(f"   F1-Score: {final_advanced_comparison.loc[best_model_idx, 'F1-Score']:.4f}")
print(f"\n   Improvement over baseline LR: {((best_model_acc - lr_accuracy) * 100):.2f}%")
if 'xgb_tuned_accuracy' in globals():
    print(f"   Improvement over Tuned XGBoost: {((best_model_acc - xgb_tuned_accuracy) * 100):.2f}%")

# Check if we reached 60%
if best_model_acc >= 0.60:
    print(f"\n🎉 SUCCESS! Achieved {best_model_acc*100:.2f}% accuracy - Above 60% target!")
else:
    print(f"\n📊 Current best: {best_model_acc*100:.2f}% - {((0.60 - best_model_acc) * 100):.2f}% away from 60% target")
    print(f"   This is still excellent performance for CSGO match prediction!")

print(f"{'='*80}")

### Summary of Improvements

**Techniques Applied:**
1. ✅ Random Forest - Ensemble method combining multiple trees
2. ✅ Feature Selection - Using only the most important features
3. ✅ Hyperparameter Tuning - Grid Search for optimal parameters
4. ✅ Additional Feature Engineering - Creating difference and interaction features

**Expected Improvements:**
- Random Forest typically improves over single Decision Tree by 2-5%
- Feature selection can improve performance by reducing noise
- Hyperparameter tuning can add 1-3% improvement
- Better feature engineering can add 2-5% improvement

**Total Potential Improvement:** 5-15% over baseline models

In [ ]:
# Compare all models
comparison_df = pd.DataFrame({
    'Model': ['Logistic Regression', 'Decision Tree', 'K-Means (ARI)'],
    'Accuracy': [lr_accuracy, dt_accuracy, ari_score],
    'Precision': [lr_precision, dt_precision, np.nan],
    'Recall': [lr_recall, dt_recall, np.nan],
    'F1-Score': [lr_f1, dt_f1, np.nan],
    'Silhouette': [np.nan, np.nan, silhouette]
})

print("Model Comparison:")
print(comparison_df.to_string(index=False))

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy comparison
axes[0].bar(comparison_df['Model'][:2], comparison_df['Accuracy'][:2], 
            color=['skyblue', 'lightgreen'])
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Model Accuracy Comparison')
axes[0].set_ylim([0, 1])
axes[0].grid(axis='y', alpha=0.3)

# Metrics comparison
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
x = np.arange(len(metrics))
width = 0.35

lr_metrics = [lr_accuracy, lr_precision, lr_recall, lr_f1]
dt_metrics = [dt_accuracy, dt_precision, dt_recall, dt_f1]

axes[1].bar(x - width/2, lr_metrics, width, label='Logistic Regression', alpha=0.8)
axes[1].bar(x + width/2, dt_metrics, width, label='Decision Tree', alpha=0.8)
axes[1].set_ylabel('Score')
axes[1].set_title('Detailed Metrics Comparison')
axes[1].set_xticks(x)
axes[1].set_xticklabels(metrics)
axes[1].legend()
axes[1].set_ylim([0, 1])
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## Final Results Summary

This section provides a comprehensive summary of all model results and key findings from the analysis.

In [ ]:
# ============================================================================
# COMPREHENSIVE RESULTS SUMMARY
# ============================================================================

print("="*80)
print("CSGO MATCH PREDICTION - COMPREHENSIVE RESULTS SUMMARY")
print("="*80)

# Dataset Information
print("\n" + "="*80)
print("1. DATASET INFORMATION")
print("="*80)
print(f"Total number of matches: {len(df_features)}")
print(f"Number of features: {X.shape[1]}")
print(f"Target variable: winner (Team 1 or Team 2)")
print(f"Target distribution:")
print(y.value_counts())
print(f"\nClass balance:")
print(y.value_counts(normalize=True))

# Data Preprocessing Summary
print("\n" + "="*80)
print("2. DATA PREPROCESSING SUMMARY")
print("="*80)
print(f"✓ Missing values handled: {df_processed.isnull().sum().sum()} missing values after imputation")
print(f"✓ Features scaled: {X_scaled.shape[1]} features normalized using StandardScaler")
print(f"✓ Feature engineering: Created {len(new_features)} new aggregated features")
print(f"  - Team average ratings, impact, KDR")
print(f"  - Rank difference, points difference, H2H advantage")

# Model Performance Summary
print("\n" + "="*80)
print("3. MODEL PERFORMANCE SUMMARY")
print("="*80)

# Create comprehensive comparison table
results_summary = pd.DataFrame({
    'Model': ['Logistic Regression', 'Decision Tree', 'K-Means Clustering'],
    'Type': ['Supervised (Classification)', 'Supervised (Classification)', 'Unsupervised (Clustering)'],
    'Accuracy/ARI': [f"{lr_accuracy:.4f}", f"{dt_accuracy:.4f}", f"{ari_score:.4f}"],
    'Precision': [f"{lr_precision:.4f}" if 'lr_precision' in globals() else "N/A", 
                  f"{dt_precision:.4f}" if 'dt_precision' in globals() else "N/A", 
                  "N/A"],
    'Recall': [f"{lr_recall:.4f}" if 'lr_recall' in globals() else "N/A", 
               f"{dt_recall:.4f}" if 'dt_recall' in globals() else "N/A", 
               "N/A"],
    'F1-Score': [f"{lr_f1:.4f}" if 'lr_f1' in globals() else "N/A", 
                 f"{dt_f1:.4f}" if 'dt_f1' in globals() else "N/A", 
                 "N/A"],
    'Silhouette Score': ["N/A", "N/A", f"{silhouette:.4f}" if 'silhouette' in globals() else "N/A"]
})

print("\n" + results_summary.to_string(index=False))

# Best Model Identification
print("\n" + "-"*80)
print("BEST MODEL IDENTIFICATION")
print("-"*80)
# Find best model from all available models
all_models = {}
if 'lr_accuracy' in globals():
    all_models['Logistic Regression'] = lr_accuracy
if 'dt_accuracy' in globals():
    all_models['Decision Tree'] = dt_accuracy
if 'rf_accuracy' in globals():
    all_models['Random Forest'] = rf_accuracy
if 'lr_selected_accuracy' in globals():
    all_models['LR with Feature Selection'] = lr_selected_accuracy
if 'rf_tuned_accuracy' in globals():
    all_models['Tuned Random Forest'] = rf_tuned_accuracy
if 'gb_accuracy' in globals():
    all_models['Gradient Boosting'] = gb_accuracy
if 'xgb_accuracy' in globals():
    all_models['XGBoost'] = xgb_accuracy
if 'xgb_tuned_accuracy' in globals():
    all_models['Tuned XGBoost'] = xgb_tuned_accuracy
if 'voting_accuracy' in globals():
    all_models['Voting Ensemble'] = voting_accuracy

if all_models:
    best_model_name = max(all_models, key=all_models.get)
    best_accuracy = all_models[best_model_name]
    print(f"🏆 Best Classification Model: {best_model_name}")
    print(f"   Accuracy: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")
    
    # Print metrics for best model
    if best_model_name == "Logistic Regression":
        print(f"   Precision: {lr_precision:.4f}")
        print(f"   Recall: {lr_recall:.4f}")
        print(f"   F1-Score: {lr_f1:.4f}")
    elif best_model_name == "Decision Tree":
        print(f"   Precision: {dt_precision:.4f}")
        print(f"   Recall: {dt_recall:.4f}")
        print(f"   F1-Score: {dt_f1:.4f}")
    elif best_model_name == "Random Forest":
        print(f"   Precision: {rf_precision:.4f}")
        print(f"   Recall: {rf_recall:.4f}")
        print(f"   F1-Score: {rf_f1:.4f}")
    elif best_model_name == "LR with Feature Selection":
        print(f"   Precision: {lr_selected_precision:.4f}")
        print(f"   Recall: {lr_selected_recall:.4f}")
        print(f"   F1-Score: {lr_selected_f1:.4f}")
    elif best_model_name == "Tuned Random Forest":
        print(f"   Precision: {rf_tuned_precision:.4f}")
        print(f"   Recall: {rf_tuned_recall:.4f}")
        print(f"   F1-Score: {rf_tuned_f1:.4f}")
    elif best_model_name == "Gradient Boosting":
        print(f"   Precision: {gb_precision:.4f}")
        print(f"   Recall: {gb_recall:.4f}")
        print(f"   F1-Score: {gb_f1:.4f}")
    elif best_model_name == "XGBoost":
        print(f"   Precision: {xgb_precision:.4f}")
        print(f"   Recall: {xgb_recall:.4f}")
        print(f"   F1-Score: {xgb_f1:.4f}")
    elif best_model_name == "Tuned XGBoost":
        print(f"   Precision: {xgb_tuned_precision:.4f}")
        print(f"   Recall: {xgb_tuned_recall:.4f}")
        print(f"   F1-Score: {xgb_tuned_f1:.4f}")
    elif best_model_name == "Voting Ensemble":
        print(f"   Precision: {voting_precision:.4f}")
        print(f"   Recall: {voting_recall:.4f}")
        print(f"   F1-Score: {voting_f1:.4f}")
    
    # Show improvement
    if 'lr_accuracy' in globals():
        improvement = ((best_accuracy - lr_accuracy) * 100)
        print(f"\n   Improvement over baseline LR: {improvement:+.2f}%")

# Key Insights
print("\n" + "="*80)
print("4. KEY INSIGHTS AND FINDINGS")
print("="*80)

if 'feature_importance' in globals():
    print("\n📊 Top 5 Most Important Features (from Decision Tree):")
    top_5_features = feature_importance.head(5)
    for idx, row in top_5_features.iterrows():
        print(f"   {idx+1}. {row['feature']}: {row['importance']:.4f}")

print("\n📈 Model Comparison Insights:")
if 'lr_accuracy' in globals() and 'dt_accuracy' in globals():
    print(f"   • Logistic Regression provides a linear baseline with {lr_accuracy*100:.2f}% accuracy")
    print(f"   • Decision Tree captures non-linear patterns with {dt_accuracy*100:.2f}% accuracy")
    if dt_accuracy > lr_accuracy:
        print(f"   • Decision Tree performs better, suggesting non-linear relationships in the data")
    else:
        print(f"   • Logistic Regression performs better, suggesting linear relationships dominate")

if 'ari_score' in globals():
    print(f"   • K-Means clustering shows {ari_score:.4f} Adjusted Rand Index with actual labels")
    if ari_score > 0.5:
        print(f"   • Good cluster separation - clusters align well with match outcomes")
    elif ari_score > 0.3:
        print(f"   • Moderate cluster separation - some alignment with match outcomes")
    else:
        print(f"   • Weak cluster separation - clusters don't strongly align with outcomes")

# Data Quality Summary
print("\n" + "="*80)
print("5. DATA QUALITY SUMMARY")
print("="*80)
print(f"✓ Original dataset shape: {df.shape}")
print(f"✓ Processed dataset shape: {df_features.shape}")
print(f"✓ Training samples: {X_train.shape[0]}")
print(f"✓ Test samples: {X_test.shape[0]}")
print(f"✓ Feature count: {X_train.shape[1]}")

# Ethical Considerations Reminder
print("\n" + "="*80)
print("6. ETHICAL CONSIDERATIONS REMINDER")
print("="*80)
print("⚠️  Key Biases Identified:")
print("   1. Temporal Bias - Meta-game changes over time")
print("   2. Team Representation Bias - Uneven team distribution")
print("   3. Ranking System Bias - Rankings may not reflect current strength")
print("   4. Data Collection Bias - Limited tournament/region coverage")
print("   5. Player Performance Bias - Individual stats vs team dynamics")
print("\n💡 Mitigation strategies documented in Section 1.1.5")

# Final Recommendations
print("\n" + "="*80)
print("7. RECOMMENDATIONS")
print("="*80)
print("✓ Use the best performing model ({}) for match predictions".format(best_model if 'best_model' in locals() else "Decision Tree"))
print("✓ Regularly retrain models with recent data to account for meta-game changes")
print("✓ Monitor model performance across different teams and time periods")
print("✓ Consider ensemble methods combining multiple models for improved accuracy")
print("✓ Validate predictions with domain experts before deployment")

print("\n" + "="*80)
print("END OF SUMMARY")
print("="*80)

### Model Performance Interpretation

**Logistic Regression:**
- Provides a linear decision boundary
- Good baseline model with interpretable coefficients
- Performance: [Interpret results]

**Decision Tree:**
- Captures non-linear relationships
- Feature importance reveals key predictors
- Performance: [Interpret results]

**K-Means Clustering:**
- Unsupervised approach to identify patterns
- Clusters may correspond to winning/losing teams
- Performance: [Interpret results]

## 1.1.5 Ethical Considerations (5%)

### Potential Biases and Fairness Issues

1. **Temporal Bias**: 
   - **Issue**: The dataset spans multiple years (2016-2017+), and the CSGO meta-game evolves over time. Models trained on older data may not generalize to current gameplay.
   - **Impact**: Predictions may favor strategies or team compositions that were effective in the past but are no longer relevant.

2. **Team Representation Bias**:
   - **Issue**: Some teams may appear more frequently in the dataset than others, leading to overfitting to specific team characteristics.
   - **Impact**: The model may perform better on frequently-seen teams and poorly on underrepresented teams.

3. **Ranking System Bias**:
   - **Issue**: World rankings may not accurately reflect current team strength, especially for teams that have recently changed rosters or strategies.
   - **Impact**: Models relying heavily on rankings may make incorrect predictions for teams in transition.

4. **Data Collection Bias**:
   - **Issue**: The dataset may only include matches from certain tournaments or regions, missing data from other competitive scenes.
   - **Impact**: The model may not generalize well to matches from underrepresented regions or tournament types.

5. **Player Performance Bias**:
   - **Issue**: Individual player statistics may not account for team chemistry, role changes, or temporary performance fluctuations.
   - **Impact**: Models may overvalue individual performance metrics while undervaluing team dynamics.

### Mitigation Strategies

1. **Temporal Validation**:
   - Implement time-based cross-validation to ensure the model performs well across different time periods.
   - Regularly retrain the model with recent data to adapt to meta-game changes.
   - Consider adding temporal features (e.g., days since last match) to account for recency effects.

2. **Balanced Sampling**:
   - Use stratified sampling to ensure balanced representation of teams in training and test sets.
   - Apply class weights during training to handle imbalanced team representation.
   - Consider oversampling underrepresented teams or undersampling overrepresented ones.

3. **Feature Engineering for Robustness**:
   - Create relative features (e.g., rank difference, rating difference) rather than absolute values to reduce bias.
   - Use rolling averages or recent performance metrics instead of historical averages.
   - Include confidence intervals or uncertainty estimates in predictions.

4. **Diverse Data Collection**:
   - Ensure the dataset includes matches from various tournaments, regions, and competitive levels.
   - Document data collection methodology and any potential gaps in coverage.
   - Consider collecting additional metadata (tournament type, map played, etc.) to improve model fairness.

5. **Model Interpretability**:
   - Use interpretable models (like decision trees) or provide feature importance analysis.
   - Conduct fairness audits to check if the model performs equally well across different team types.
   - Implement model monitoring to detect performance degradation or bias introduction over time.

6. **Transparency and Documentation**:
   - Clearly document model limitations and assumptions.
   - Provide confidence scores with predictions to indicate uncertainty.
   - Regularly review and update the model based on new data and feedback.

7. **Ethical Use Guidelines**:
   - Ensure predictions are used responsibly and not to manipulate betting markets.
   - Consider the impact on players and teams if predictions are made public.
   - Respect privacy of player performance data and comply with data protection regulations.

## Summary

This notebook has implemented a complete machine learning workflow for CSGO match prediction:

1. **Dataset Selection**: CSGO match dataset with comprehensive team and player statistics
2. **Data Preprocessing**: Handled missing values, feature engineering, scaling, and encoding
3. **Exploratory Data Analysis**: Visualizations and pattern identification
4. **Model Development**: Trained Logistic Regression, Decision Tree, and K-Means models
5. **Evaluation**: Comprehensive metrics and performance comparison
6. **Ethical Considerations**: Identified biases and proposed mitigation strategies

The models demonstrate [summarize performance] and can be used for match outcome prediction with appropriate considerations for the identified biases.